<a href="https://colab.research.google.com/github/Kaiziferr/deep_learning_workshop/blob/master/multilayer_perceptron/07_workshop_learningrate_adaptative.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

import tensorflow 

from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler

from tensorflow.keras.metrics import Recall

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import math


In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/ionosphere.csv')
data_values = data.values

# Split data

---



In [3]:
X = data_values[:,0:34].astype('float')
y = data_values[:,34]

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.75, shuffle=True) 

In [5]:
# Encoder
encoder = LabelEncoder()
y_train_encoder = encoder.fit_transform(y_train)
y_test_encoder = encoder.transform(y_test)

In [6]:
# Escalar
estimator_scaler = StandardScaler()
X_train_scaler = estimator_scaler.fit_transform(X_train)
X_test_scaler = estimator_scaler.transform(X_test)

In [7]:
data['g'].value_counts()

g    224
b    126
Name: g, dtype: int64

# Optimizador Adaptativo Adam

---



In [8]:
model = Sequential()
model.add(Dense(34, input_dim = 34, activation='relu', kernel_initializer='glorot_uniform'))
model.add(Dense(1,activation='sigmoid', kernel_initializer='glorot_uniform'))

In [9]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', Recall()])
model.fit(X_train_scaler, y_train_encoder, validation_data=(X_test_scaler, y_test_encoder), epochs=50, batch_size=28, verbose = 1)

Epoch 1/50
10/10 [==============================] - 1s 34ms/step - loss: 0.8087 - accuracy: 0.3931 - recall: 0.4068 - val_loss: 0.7856 - val_accuracy: 0.5455 - val_recall: 0.6383
Epoch 2/50
10/10 [==============================] - 0s 9ms/step - loss: 0.6661 - accuracy: 0.6870 - recall: 0.8192 - val_loss: 0.6882 - val_accuracy: 0.6364 - val_recall: 0.8298
Epoch 3/50
10/10 [==============================] - 0s 7ms/step - loss: 0.5613 - accuracy: 0.7939 - recall: 0.9379 - val_loss: 0.6259 - val_accuracy: 0.7045 - val_recall: 0.9149
Epoch 4/50
10/10 [==============================] - 0s 5ms/step - loss: 0.4955 - accuracy: 0.8168 - recall: 0.9548 - val_loss: 0.5777 - val_accuracy: 0.7159 - val_recall: 0.9362
Epoch 5/50
10/10 [==============================] - 0s 8ms/step - loss: 0.4449 - accuracy: 0.8435 - recall: 0.9605 - val_loss: 0.5430 - val_accuracy: 0.7500 - val_recall: 0.9362
Epoch 6/50
10/10 [==============================] - 0s 5ms/step - loss: 0.4052 - accuracy: 0.8588 - recall: 0

# **Planificación de learning rate en función del tiempo**

---



In [10]:
model = Sequential()
model.add(Dense(34, input_dim = 34, activation='relu', kernel_initializer='glorot_uniform'))
model.add(Dense(1, activation='sigmoid', kernel_initializer='glorot_uniform'))

In [11]:
epoch = 50
lr = 0.1
decay_rate = lr/epoch
momentum = 0.8

sgd = SGD(learning_rate=lr, momentum=momentum, decay = decay_rate, nesterov=False)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy' , Recall()])

In [12]:
model.fit(X_train_scaler, y_train_encoder, validation_data=(X_test_scaler, y_test_encoder), epochs=epoch, batch_size=28, verbose = 1)

Epoch 1/50
10/10 [==============================] - 1s 34ms/step - loss: 0.5144 - accuracy: 0.7710 - recall_1: 0.7797 - val_loss: 0.4623 - val_accuracy: 0.8295 - val_recall_1: 0.9574
Epoch 2/50
10/10 [==============================] - 0s 4ms/step - loss: 0.2615 - accuracy: 0.8931 - recall_1: 0.9322 - val_loss: 0.3090 - val_accuracy: 0.8523 - val_recall_1: 0.9574
Epoch 3/50
10/10 [==============================] - 0s 4ms/step - loss: 0.1891 - accuracy: 0.9122 - recall_1: 0.9605 - val_loss: 0.2439 - val_accuracy: 0.8864 - val_recall_1: 0.9787
Epoch 4/50
10/10 [==============================] - 0s 6ms/step - loss: 0.1466 - accuracy: 0.9389 - recall_1: 0.9605 - val_loss: 0.2345 - val_accuracy: 0.9205 - val_recall_1: 1.0000
Epoch 5/50
10/10 [==============================] - 0s 6ms/step - loss: 0.1222 - accuracy: 0.9542 - recall_1: 0.9887 - val_loss: 0.2165 - val_accuracy: 0.9318 - val_recall_1: 1.0000
Epoch 6/50
10/10 [==============================] - 0s 7ms/step - loss: 0.1038 - accuracy

# **Tasa de aprendizaje basado en caidas (drop)**

---



In [13]:
def step_decay(epochs):
  initial_rate = 0.1
  drop = 0.5
  epochs_drop = 10
  lrate = initial_rate * math.pow(drop, math.floor(1+epochs) / epochs_drop)
  print(lrate, epochs)
  return lrate

In [14]:
model = Sequential()
model.add(Dense(34, input_dim = 34, activation='relu', kernel_initializer='glorot_uniform'))
model.add(Dense(1, activation='sigmoid'))

In [15]:
sgd = SGD(learning_rate=0.0, momentum = 0.9)
model.compile(loss = 'binary_crossentropy', optimizer=sgd, metrics = ['accuracy',Recall()])

In [16]:
lrate = LearningRateScheduler(step_decay)
callback_list = [lrate]

In [17]:
model.fit(X_train_scaler, y_train_encoder, validation_data=(X_test_scaler, y_test_encoder), epochs=50, batch_size=25, callbacks=callback_list, verbose=1)

0.09330329915368074 0
Epoch 1/50
11/11 [==============================] - 1s 32ms/step - loss: 0.5500 - accuracy: 0.7405 - recall_2: 0.8362 - val_loss: 0.2970 - val_accuracy: 0.8636 - val_recall_2: 0.9362 - lr: 0.0933
0.08705505632961241 1
Epoch 2/50
11/11 [==============================] - 0s 4ms/step - loss: 0.2610 - accuracy: 0.8855 - recall_2: 0.8927 - val_loss: 0.2472 - val_accuracy: 0.8977 - val_recall_2: 0.9574 - lr: 0.0871
0.08122523963562356 2
Epoch 3/50
11/11 [==============================] - 0s 5ms/step - loss: 0.1608 - accuracy: 0.9466 - recall_2: 0.9605 - val_loss: 0.3011 - val_accuracy: 0.8864 - val_recall_2: 0.9787 - lr: 0.0812
0.0757858283255199 3
Epoch 4/50
11/11 [==============================] - 0s 5ms/step - loss: 0.1105 - accuracy: 0.9580 - recall_2: 0.9831 - val_loss: 0.3176 - val_accuracy: 0.9091 - val_recall_2: 1.0000 - lr: 0.0758
0.07071067811865477 4
Epoch 5/50
11/11 [==============================] - 0s 6ms/step - loss: 0.0865 - accuracy: 0.9771 - recall_2: 

In [18]:
model.fit(X_train_scaler, y_train_encoder, validation_data=(X_test_scaler, y_test_encoder), epochs=50, batch_size=25, callbacks=callback_list
          , verbose = 1)

0.09330329915368074 0
Epoch 1/50
11/11 [==============================] - 0s 9ms/step - loss: 0.0106 - accuracy: 1.0000 - recall_2: 1.0000 - val_loss: 0.4331 - val_accuracy: 0.9205 - val_recall_2: 1.0000 - lr: 0.0933
0.08705505632961241 1
Epoch 2/50
11/11 [==============================] - 0s 6ms/step - loss: 0.0107 - accuracy: 1.0000 - recall_2: 1.0000 - val_loss: 0.4404 - val_accuracy: 0.9205 - val_recall_2: 1.0000 - lr: 0.0871
0.08122523963562356 2
Epoch 3/50
11/11 [==============================] - 0s 6ms/step - loss: 0.0096 - accuracy: 1.0000 - recall_2: 1.0000 - val_loss: 0.4361 - val_accuracy: 0.9205 - val_recall_2: 1.0000 - lr: 0.0812
0.0757858283255199 3
Epoch 4/50
11/11 [==============================] - 0s 7ms/step - loss: 0.0096 - accuracy: 1.0000 - recall_2: 1.0000 - val_loss: 0.4395 - val_accuracy: 0.9205 - val_recall_2: 1.0000 - lr: 0.0758
0.07071067811865477 4
Epoch 5/50
11/11 [==============================] - 0s 7ms/step - loss: 0.0084 - accuracy: 1.0000 - recall_2: 1

- Incrementar la tasa de aprendizaje inicial, debido a que la tasa de aprendizaje disminuira, inicie con un valor mayor desde el cual disminuir

- Utilizar un gran momentum

- Experimente con diferentes planificacion